# Mais sobre `SELECT`

- temporary tables
- views
- variables
- subqueries


In [1]:
import mysql.connector
from functools import partial

def get_connection_helper():
    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            print('Executando query:')
            cursor.execute(query, args)
            for result in cursor:
                print(result)

    connection = mysql.connector.connect(
        host='localhost',
        user='megadados',
        password='megadados2020',
        database='sakila'
    )
    return connection, partial(run_db_query, connection)

connection, db = get_connection_helper()

## Aquecimento

Quanta receita foi gerada para cada categoria de filmes? Liste do maior para o menor.

In [2]:
db('''
SELECT 
    name, sum(amount) as total
FROM
    payment
    INNER JOIN rental USING (rental_id)
    INNER JOIN inventory USING (inventory_id)
    INNER JOIN film_category USING (film_id)
    INNER JOIN category USING (category_id)
GROUP BY category_id
ORDER BY total DESC;
''')

Executando query:
('Sports', Decimal('5314.21'))
('Sci-Fi', Decimal('4756.98'))
('Animation', Decimal('4656.30'))
('Drama', Decimal('4587.39'))
('Comedy', Decimal('4383.58'))
('Action', Decimal('4375.85'))
('New', Decimal('4351.62'))
('Games', Decimal('4281.33'))
('Foreign', Decimal('4270.67'))
('Family', Decimal('4226.07'))
('Documentary', Decimal('4217.52'))
('Horror', Decimal('3722.54'))
('Children', Decimal('3655.55'))
('Classics', Decimal('3639.59'))
('Travel', Decimal('3549.64'))
('Music', Decimal('3417.72'))


Liste os filmes e o numero de vezes em que foram alugados

In [3]:
db('''
SELECT 
    title, COUNT(rental_id) as cnt
FROM
    film
    LEFT OUTER JOIN inventory USING (film_id)
    LEFT OUTER JOIN rental USING (inventory_id)
GROUP BY
    film_id
ORDER BY
    cnt ASC
LIMIT 30
''')

Executando query:
('ARK RIDGEMONT', 0)
('FRANKENSTEIN STRANGER', 0)
('PEARL DESTINY', 0)
('FIREHOUSE VIETNAM', 0)
('FLOATS GARDEN', 0)
('GUMP DATE', 0)
('CATCH AMISTAD', 0)
('DELIVERANCE MULHOLLAND', 0)
('ORDER BETRAYED', 0)
('ARGONAUTS TOWN', 0)
('BOONDOCK BALLROOM', 0)
('APOLLO TEEN', 0)
('HATE HANDICAP', 0)
('ALICE FANTASIA', 0)
('GLADIATOR WESTWARD', 0)
('COMMANDMENTS EXPRESS', 0)
('KILL BROTHERHOOD', 0)
('CROSSING DIVORCE', 0)
('CROWDS TELEMARK', 0)
('CHOCOLATE DUCK', 0)
('PSYCHO SHRUNK', 0)
('BUTCH PANTHER', 0)
('CRYSTAL BREAKING', 0)
('MUPPET MILE', 0)
('ARSENIC INDEPENDENCE', 0)
('STONED PUNK', 0)
('CHINATOWN GLADIATOR', 0)
('PERDITION FARGO', 0)
('KENTUCKIAN GIANT', 0)
('HOCUS FRIDA', 0)


Semana do "DAN HARRIS": liste os clientes que nunca assistiram um filme do ator "DAN HARRIS" ou que já assistiram mas onde a ultima vez em que assistiram um filme dele foi antes de '2005-06-01'

In [4]:
# Quem ja assistiu filme do DAN HARRIS.
db('''
DROP TABLE IF EXISTS ja_vi_dan_harris
''')

db('''
CREATE TEMPORARY TABLE ja_vi_dan_harris AS
SELECT 
	customer_id, MAX(rental_date) as last_rental
FROM
    customer
	LEFT OUTER JOIN rental USING (customer_id)
    LEFT OUTER JOIN inventory USING (inventory_id)
	LEFT OUTER JOIN film USING (film_id)
	LEFT OUTER JOIN film_actor USING (film_id)
	LEFT OUTER JOIN actor USING (actor_id)
WHERE 
    actor.first_name = "Dan" AND actor.last_name = "Harris"
GROUP BY 
    customer_id
ORDER BY 
    customer_id, last_rental ASC;
''')

db('''
SELECT * from ja_vi_dan_harris LIMIT 20
''')

Executando query:
Executando query:
Executando query:
(2, datetime.datetime(2005, 7, 29, 0, 12, 59))
(4, datetime.datetime(2005, 8, 23, 7, 43))
(6, datetime.datetime(2005, 8, 2, 18, 55, 15))
(7, datetime.datetime(2005, 8, 20, 5, 53, 31))
(8, datetime.datetime(2005, 8, 1, 13, 5, 35))
(10, datetime.datetime(2005, 8, 18, 9, 19, 12))
(14, datetime.datetime(2005, 7, 9, 6, 37, 7))
(15, datetime.datetime(2005, 8, 18, 14, 46, 26))
(17, datetime.datetime(2005, 7, 12, 20, 52, 41))
(18, datetime.datetime(2005, 7, 8, 10, 15, 38))
(19, datetime.datetime(2005, 5, 25, 18, 43, 49))
(23, datetime.datetime(2006, 2, 14, 15, 16, 3))
(25, datetime.datetime(2005, 7, 30, 10, 12, 1))
(26, datetime.datetime(2005, 7, 29, 0, 33, 36))
(27, datetime.datetime(2005, 8, 17, 5, 25, 57))
(29, datetime.datetime(2005, 8, 18, 15, 20, 46))
(30, datetime.datetime(2005, 8, 23, 4, 17, 56))
(31, datetime.datetime(2005, 6, 19, 10, 6, 20))
(36, datetime.datetime(2005, 7, 29, 5, 52, 50))
(39, datetime.datetime(2005, 7, 31, 19, 15

In [5]:
# Quem nunca viu Dan Harris.
db('''
DROP TABLE IF EXISTS nunca_vi_dan_harris
''')

db('''
CREATE TEMPORARY TABLE nunca_vi_dan_harris AS
SELECT 
    customer.customer_id
FROM 
    customer 
    LEFT OUTER JOIN ja_vi_dan_harris USING (customer_id)
WHERE
    ja_vi_dan_harris.customer_id IS NULL    
''')

db('''
SELECT * FROM nunca_vi_dan_harris LIMIT 20
''')

Executando query:
Executando query:
Executando query:
(1,)
(3,)
(5,)
(12,)
(21,)
(22,)
(28,)
(32,)
(37,)
(38,)
(41,)
(44,)
(48,)
(50,)
(52,)
(56,)
(58,)
(59,)
(60,)
(62,)


In [6]:
# Quem faz tempo que não ve o Dan Harris.
db('''
DROP TABLE IF EXISTS faz_tempo_sem_dan_harris
''')

db('''
CREATE TEMPORARY TABLE faz_tempo_sem_dan_harris AS
SELECT
    customer_id
FROM
    ja_vi_dan_harris
WHERE
    last_rental < '2005-06-01'
''')

db ('''
SELECT * FROM faz_tempo_sem_dan_harris LIMIT 20
''')

Executando query:
Executando query:
Executando query:
(19,)
(102,)
(176,)
(185,)
(210,)
(235,)
(270,)
(307,)
(375,)
(468,)
(511,)
(517,)
(526,)
(552,)


In [7]:
# Junta as listas.
db('''
DROP TABLE IF EXISTS tasca_dan_harris_neles
''')

db('''
CREATE TEMPORARY TABLE tasca_dan_harris_neles AS
SELECT * FROM nunca_vi_dan_harris UNION SELECT * FROM faz_tempo_sem_dan_harris
''')

db('''
SELECT 
    first_name, last_name 
FROM 
    tasca_dan_harris_neles 
    INNER JOIN customer USING (customer_id) 
ORDER BY first_name, last_name ASC
''')

db('''
DROP TABLE ja_vi_dan_harris
''')

db('''
DROP TABLE nunca_vi_dan_harris
''')

db('''
DROP TABLE faz_tempo_sem_dan_harris
''')

db('''
DROP TABLE tasca_dan_harris_neles
''')

Executando query:
Executando query:
Executando query:
('AARON', 'SELBY')
('AGNES', 'BISHOP')
('ALAN', 'KAHN')
('ALBERT', 'CROUSE')
('ALICIA', 'MILLS')
('ALLAN', 'CORNISH')
('ALLEN', 'BUTTERFIELD')
('ALLISON', 'STANLEY')
('AMANDA', 'CARTER')
('AMY', 'LOPEZ')
('ANA', 'BRADLEY')
('ANDREA', 'HENDERSON')
('ANDREW', 'PURDY')
('ANGEL', 'BARCLAY')
('ANITA', 'MORALES')
('ANN', 'EVANS')
('ANNA', 'HILL')
('ANNE', 'POWELL')
('ANNETTE', 'OLSON')
('ANNIE', 'RUSSELL')
('ANTHONY', 'SCHWAB')
('ANTONIO', 'MEEK')
('ARMANDO', 'GRUBER')
('AUDREY', 'RAY')
('AUSTIN', 'CINTRON')
('BEATRICE', 'ARNOLD')
('BECKY', 'MILES')
('BEN', 'EASTER')
('BERNARD', 'COLBY')
('BESSIE', 'MORRISON')
('BETH', 'FRANKLIN')
('BILLIE', 'HORTON')
('BILLY', 'POULIN')
('BOB', 'PFEIFFER')
('BRAD', 'MCCURDY')
('BRANDY', 'GRAVES')
('BRENT', 'HARKINS')
('BRETT', 'CORNWELL')
('BRIAN', 'WYMAN')
('BRITTANY', 'RILEY')
('BYRON', 'BOX')
('CALVIN', 'MARTEL')
('CARLA', 'GUTIERREZ')
('CAROLE', 'BARNETT')
('CARRIE', 'PORTER')
('CASEY', 'MENA')
('CAT

## Views

Uma *view* é uma tabela virtual, construida a partir de um comando `SELECT`. Por exemplo: execute o código a seguir no MySQL Workbench.

```SQL
USE sakila;

DROP VIEW IF EXISTS movie_count;

CREATE VIEW movie_count AS
	SELECT 
		title, COUNT(rental_id) as cnt
	FROM
		film
		LEFT OUTER JOIN inventory USING (film_id)
		LEFT OUTER JOIN rental USING (inventory_id)
	GROUP BY
		film_id
	ORDER BY
		cnt ASC;

```

Agora temos uma *view* chamada `movie_count`. Vamos verificar que ela funcionou, listando as 30 primeiras linhas:

In [8]:
db('''
SELECT * FROM movie_count LIMIT 100
''')

Executando query:
('ALICE FANTASIA', 0)
('APOLLO TEEN', 0)
('ARGONAUTS TOWN', 0)
('ARK RIDGEMONT', 0)
('ARSENIC INDEPENDENCE', 0)
('BOONDOCK BALLROOM', 0)
('BUTCH PANTHER', 0)
('CATCH AMISTAD', 0)
('CHINATOWN GLADIATOR', 0)
('CHOCOLATE DUCK', 0)
('COMMANDMENTS EXPRESS', 0)
('CROSSING DIVORCE', 0)
('CROWDS TELEMARK', 0)
('CRYSTAL BREAKING', 0)
('STONED PUNK', 0)
('DELIVERANCE MULHOLLAND', 0)
('FIREHOUSE VIETNAM', 0)
('FLOATS GARDEN', 0)
('FRANKENSTEIN STRANGER', 0)
('GLADIATOR WESTWARD', 0)
('GUMP DATE', 0)
('HATE HANDICAP', 0)
('HOCUS FRIDA', 0)
('KENTUCKIAN GIANT', 0)
('KILL BROTHERHOOD', 0)
('MUPPET MILE', 0)
('ORDER BETRAYED', 0)
('PEARL DESTINY', 0)
('PERDITION FARGO', 0)
('PSYCHO SHRUNK', 0)
('RAIDERS ANTITRUST', 0)
('RAINBOW SHOCK', 0)
('ROOF CHAMPION', 0)
('SISTER FREDDY', 0)
('SKY MIRACLE', 0)
('SUICIDES SILENCE', 0)
('TADPOLE PARK', 0)
('TREASURE COMMAND', 0)
('VILLAIN DESPERATE', 0)
('VOLUME HOUSE', 0)
('WAKE JAWS', 0)
('WALLS ARTIST', 0)
('HARDLY ROBBERS', 4)
('MIXED DOORS',

Agora suponha que alteramos a tabela `film`, mudando o nome do filme "DAZED PUNK" para "STONED PUNK".

**Atividade**: Do it.

In [9]:
db('''
UPDATE film SET title = 'STONED PUNK' WHERE title = 'DAZED PUNK'
''')

Executando query:


Verifique agora a nossa *view*:

In [10]:
db('''
SELECT * FROM movie_count LIMIT 30
''')

Executando query:
('ALICE FANTASIA', 0)
('APOLLO TEEN', 0)
('ARGONAUTS TOWN', 0)
('ARK RIDGEMONT', 0)
('ARSENIC INDEPENDENCE', 0)
('BOONDOCK BALLROOM', 0)
('BUTCH PANTHER', 0)
('CATCH AMISTAD', 0)
('CHINATOWN GLADIATOR', 0)
('CHOCOLATE DUCK', 0)
('COMMANDMENTS EXPRESS', 0)
('CROSSING DIVORCE', 0)
('CROWDS TELEMARK', 0)
('CRYSTAL BREAKING', 0)
('STONED PUNK', 0)
('DELIVERANCE MULHOLLAND', 0)
('FIREHOUSE VIETNAM', 0)
('FLOATS GARDEN', 0)
('FRANKENSTEIN STRANGER', 0)
('GLADIATOR WESTWARD', 0)
('GUMP DATE', 0)
('HATE HANDICAP', 0)
('HOCUS FRIDA', 0)
('KENTUCKIAN GIANT', 0)
('KILL BROTHERHOOD', 0)
('MUPPET MILE', 0)
('ORDER BETRAYED', 0)
('PEARL DESTINY', 0)
('PERDITION FARGO', 0)
('PSYCHO SHRUNK', 0)


Como você pode ver, as views são tabelas virtuais que são automaticamente atualizadas quando as tabelas originais são modificadas.

In [11]:
connection.rollback()

## Tabelas temporárias

Tabelas temporárias podem ser criadas para ajudar nas tarefas de manipulação de dados. Essas tabelas existem apenas pela duração da sessão. Para criar uma tabela temporária, basta adicionar a palavra-chave `TEMPORARY` no momento da criação.

É comum criar tabelas temporárias à partir do resultado de comandos `SELECT`. Por exemplo, a seguinte query permite montar uma tabela temporária com os filmes que duram mais que 3 horas:

In [12]:
db('''
CREATE TEMPORARY TABLE long_film 
SELECT
    *
FROM
    film
WHERE
    film.length > 180;
''')

Executando query:


Podemos verificar que a tabela `long_film` agora existe:

In [13]:
db('DESCRIBE long_film')

Executando query:
('film_id', 'smallint unsigned', 'NO', '', '0', 'NULL')
('title', 'varchar(255)', 'NO', '', None, 'NULL')
('description', 'text', 'YES', '', None, 'NULL')
('release_year', 'year', 'YES', '', None, 'NULL')
('language_id', 'tinyint unsigned', 'NO', '', None, 'NULL')
('original_language_id', 'tinyint unsigned', 'YES', '', None, 'NULL')
('rental_duration', 'tinyint unsigned', 'NO', '', '3', 'NULL')
('rental_rate', 'decimal(4,2)', 'NO', '', '4.99', 'NULL')
('length', 'smallint unsigned', 'YES', '', None, 'NULL')
('replacement_cost', 'decimal(5,2)', 'NO', '', '19.99', 'NULL')
('rating', "enum('G','PG','PG-13','R','NC-17')", 'YES', '', 'G', 'NULL')
('special_features', "set('Trailers','Commentaries','Deleted Scenes','Behind the Scenes')", 'YES', '', None, 'NULL')
('last_update', 'timestamp', 'NO', '', 'CURRENT_TIMESTAMP', 'on update CURRENT_TIMESTAMP')


Muito embora ela não apareça na lista de tabelas: isso é um bug do MySQL. (https://dev.mysql.com/worklog/task/?id=648)

In [14]:
db('SHOW TABLES')

Executando query:
('actor',)
('actor_info',)
('address',)
('category',)
('city',)
('country',)
('customer',)
('customer_list',)
('film',)
('film_actor',)
('film_category',)
('film_list',)
('film_text',)
('inventory',)
('language',)
('movie_count',)
('nicer_but_slower_film_list',)
('payment',)
('rental',)
('sales_by_film_category',)
('sales_by_store',)
('staff',)
('staff_list',)
('store',)


Vamos listar o conteudo desta tabela:

In [15]:
db('SELECT title FROM long_film')

Executando query:
('ANALYZE HOOSIERS',)
('BAKED CLEOPATRA',)
('CATCH AMISTAD',)
('CHICAGO NORTH',)
('CONSPIRACY SPIRIT',)
('CONTROL ANTHEM',)
('CRYSTAL BREAKING',)
('DARN FORRESTER',)
('FRONTIER CABIN',)
('GANGS PRIDE',)
('HAUNTING PIANIST',)
('HOME PITY',)
('HOTEL HAPPINESS',)
('INTRIGUE WORST',)
('JACKET FRISCO',)
('KING EVOLUTION',)
('LAWLESS VISION',)
('LOVE SUICIDES',)
('MONSOON CAUSE',)
('MOONWALKER FOOL',)
('MUSCLE BRIGHT',)
('POND SEATTLE',)
('RECORDS ZORRO',)
('REDS POCUS',)
('RUNAWAY TENENBAUMS',)
('SATURN NAME',)
('SCALAWAG DUCK',)
('SEARCHERS WAIT',)
('SMOOCHY CONTROL',)
('SOLDIERS EVOLUTION',)
('SONS INTERVIEW',)
('SORORITY QUEEN',)
('STAR OPERATION',)
('SWEET BROTHERHOOD',)
('THEORY MERMAID',)
('WIFE TURN',)
('WILD APOLLO',)
('WORST BANGER',)
('YOUNG LANGUAGE',)


Vamos apagar a tabela `long_film`:

In [16]:
db('DROP TABLE long_film')

Executando query:


### Vamos praticar

- Crie uma tabela temporária `max_duration` que contém a duração máxima de filme para cada categoria

In [17]:
db('''
DROP TABLE IF EXISTS max_duration;
''')

db('''
CREATE TEMPORARY TABLE 
    max_duration as
SELECT
    category_id,
    name,
    max(film.length) AS duration
FROM
    film
    INNER JOIN film_category USING (film_id)
    INNER JOIN category USING (category_id)
GROUP BY 
    category_id;
''')

Executando query:
Executando query:


 - Verifique a tabela.

In [18]:
db('SELECT * FROM max_duration')

Executando query:
(1, 'Action', 185)
(2, 'Animation', 185)
(3, 'Children', 178)
(4, 'Classics', 184)
(5, 'Comedy', 185)
(6, 'Documentary', 183)
(7, 'Drama', 181)
(8, 'Family', 184)
(9, 'Foreign', 184)
(10, 'Games', 185)
(11, 'Horror', 181)
(12, 'Music', 185)
(13, 'New', 183)
(14, 'Sci-Fi', 185)
(15, 'Sports', 184)
(16, 'Travel', 185)


- Agora use a tabela temporária para construir uma tabela com as categorias e seus respectivos filmes mais longos:

In [19]:
db('''
SELECT 
    name, 
    title
FROM
    max_duration
    INNER JOIN film_category USING (category_id)
    INNER JOIN film USING (film_id)
WHERE
    film.length = duration
ORDER BY name
''')

Executando query:
('Action', 'DARN FORRESTER')
('Action', 'WORST BANGER')
('Animation', 'GANGS PRIDE')
('Animation', 'POND SEATTLE')
('Children', 'FURY MURDER')
('Children', 'WRONG BEHAVIOR')
('Classics', 'CONSPIRACY SPIRIT')
('Comedy', 'CONTROL ANTHEM')
('Documentary', 'WIFE TURN')
('Documentary', 'YOUNG LANGUAGE')
('Drama', 'JACKET FRISCO')
('Family', 'KING EVOLUTION')
('Foreign', 'CRYSTAL BREAKING')
('Foreign', 'SORORITY QUEEN')
('Games', 'CHICAGO NORTH')
('Horror', 'ANALYZE HOOSIERS')
('Horror', 'LOVE SUICIDES')
('Music', 'HOME PITY')
('New', 'FRONTIER CABIN')
('Sci-Fi', 'SOLDIERS EVOLUTION')
('Sports', 'SMOOCHY CONTROL')
('Travel', 'MUSCLE BRIGHT')
('Travel', 'SWEET BROTHERHOOD')


- delete a tabela temporária

In [20]:
db('DROP TABLE IF EXISTS max_duration')

Executando query:


### Desafio!

- Gere uma tabela contendo, para cada ator, a seguinte informação:

| first_name | last_name | filmes por categoria |
|--|--|--|
| PENELOPE | GUINESS | Animation: ANACONDA CONFESSIONS; Children: LANGUAGE COWBOY; Classics: COLOR PHILADELPHIA, WESTWARD SEABISCUIT; Comedy: VERTIGO NORTHWEST; Documentary: ACADEMY DINOSAUR; Family: KING EVOLUTION, SPLASH GUMP; Foreign: MULHOLLAND BEAST; Games: BULWORTH COMMANDMENTS, HUMAN GRAFFITI; Horror: ELEPHANT TROJAN, LADY STAGE, RULES HUMAN; Music: WIZARD COLDBLOODED; New: ANGELS LIFE, OKLAHOMA JUMANJI; Sci-Fi: CHEAPER CLYDE; Sports: GLEAMING JAWBREAKER |
| NICK | WAHLBERG | Action: BULL SHAWSHANK; Animation: FIGHT JAWBREAKER; Children: JERSEY SASSY; Classics: DRACULA CRYSTAL, GILBERT PELICAN; Comedy: MALLRATS UNITED, RUSHMORE MERMAID; Documentary: ADAPTATION HOLES; Drama: WARDROBE PHANTOM; Family: APACHE DIVINE, CHISUM BEHAVIOR, INDIAN LOVE, MAGUIRE APACHE; Foreign: BABY HALL, HAPPINESS UNITED; Games: ROOF CHAMPION; Music: LUCKY FLYING; New: DESTINY SATURDAY, FLASH WARS, JEKYLL FROGMEN, MASK PEACH; Sci-Fi: CHAINSAW UPTOWN, GOODFELLAS SALUTE; Travel: LIAISONS SWEET, SMILE EARRING |
| etc | etc | etc |

Dica: use `GROUP_CONCAT` para agrupar todas as strings de uma coluna em uma string só, e `CONCAT` para unir strings particulares.

In [21]:
db('''
DROP TABLE IF EXISTS film_per_actor_category;
''')
db('''
DROP TABLE IF EXISTS film_per_actor;
''')

db('''
CREATE TEMPORARY TABLE film_per_actor_category
SELECT 
    actor_id, category_id, first_name, last_name, name,
    GROUP_CONCAT(title
        ORDER BY title ASC
        SEPARATOR ', ') as film_list
FROM
    film
    INNER JOIN film_category USING (film_id)
    INNER JOIN film_actor USING (film_id)
    INNER JOIN category USING (category_id)
    INNER JOIN actor USING (actor_id)
GROUP BY actor_id, category_id;
''')

db('''
CREATE TEMPORARY TABLE film_per_actor
SELECT 
    first_name,
    last_name,
    GROUP_CONCAT(
        CONCAT(name, ': ', film_list)
        SEPARATOR '; '
    ) AS film_category_list
FROM
    film_per_actor_category
GROUP BY actor_id;
''')


Executando query:
Executando query:
Executando query:
Executando query:


In [22]:
db('SELECT * FROM film_per_actor')

Executando query:
('PENELOPE', 'GUINESS', 'Animation: ANACONDA CONFESSIONS; Children: LANGUAGE COWBOY; Classics: COLOR PHILADELPHIA, WESTWARD SEABISCUIT; Comedy: VERTIGO NORTHWEST; Documentary: ACADEMY DINOSAUR; Family: KING EVOLUTION, SPLASH GUMP; Foreign: MULHOLLAND BEAST; Games: BULWORTH COMMANDMENTS, HUMAN GRAFFITI; Horror: ELEPHANT TROJAN, LADY STAGE, RULES HUMAN; Music: WIZARD COLDBLOODED; New: ANGELS LIFE, OKLAHOMA JUMANJI; Sci-Fi: CHEAPER CLYDE; Sports: GLEAMING JAWBREAKER')
('NICK', 'WAHLBERG', 'Action: BULL SHAWSHANK; Animation: FIGHT JAWBREAKER; Children: JERSEY SASSY; Classics: DRACULA CRYSTAL, GILBERT PELICAN; Comedy: MALLRATS UNITED, RUSHMORE MERMAID; Documentary: ADAPTATION HOLES; Drama: WARDROBE PHANTOM; Family: APACHE DIVINE, CHISUM BEHAVIOR, INDIAN LOVE, MAGUIRE APACHE; Foreign: BABY HALL, HAPPINESS UNITED; Games: ROOF CHAMPION; Music: LUCKY FLYING; New: DESTINY SATURDAY, FLASH WARS, JEKYLL FROGMEN, MASK PEACH; Sci-Fi: CHAINSAW UPTOWN, GOODFELLAS SALUTE; Travel: LIAIS

In [23]:
db('''
DROP TABLE IF EXISTS film_per_actor_category;
''')
db('''
DROP TABLE IF EXISTS film_per_actor;
''')

Executando query:
Executando query:


## Variáveis

Podemos montar uma query que retorne um valor só e armazenar este valor em uma variável, para uso posterior em outras queries. Para isso vamos usar o prefixo '@' para indicar variáveis, e o comando `SELECT ... INTO`.

Exemplo: quais são os filmes "caros" da nossa base sakila? Vamos descobrir quais filmes custam mais que um desvio padrão acima da média de preços de locação.

Primeiro vamos calcular a média e o desvio padrão dos preços de aluguel:

In [24]:
db('''
SELECT 
    AVG(rental_rate), 
    STDDEV(rental_rate)
INTO 
    @avg_rate, 
    @stddev_rate 
FROM
    film;
''')

Executando query:


Note que a query não retorna um resultado: o resultado foi armazenado direto nas variáveis `@avg_rate` e `@stddev_rate`. Vamos usar um `SELECT` sem tabelas para ver o resultado:

In [25]:
db('SELECT @avg_rate, @stddev_rate')

Executando query:
(Decimal('2.980000000'), 1.6455698101265719)


Agora podemos selecionar os filmes caros!

In [26]:
db('''
SELECT 
    title, rental_rate
FROM
    film
WHERE
    rental_rate > @avg_rate + @stddev_rate;
''')

Executando query:
('ACE GOLDFINGER', Decimal('4.99'))
('AIRPLANE SIERRA', Decimal('4.99'))
('AIRPORT POLLOCK', Decimal('4.99'))
('ALADDIN CALENDAR', Decimal('4.99'))
('ALI FOREVER', Decimal('4.99'))
('AMELIE HELLFIGHTERS', Decimal('4.99'))
('AMERICAN CIRCUS', Decimal('4.99'))
('ANTHEM LUKE', Decimal('4.99'))
('APACHE DIVINE', Decimal('4.99'))
('APOCALYPSE FLAMINGOS', Decimal('4.99'))
('ATTACKS HATE', Decimal('4.99'))
('ATTRACTION NEWTON', Decimal('4.99'))
('AUTUMN CROW', Decimal('4.99'))
('BABY HALL', Decimal('4.99'))
('BACKLASH UNDEFEATED', Decimal('4.99'))
('BEAST HUNCHBACK', Decimal('4.99'))
('BEAUTY GREASE', Decimal('4.99'))
('BEHAVIOR RUNAWAY', Decimal('4.99'))
('BETRAYED REAR', Decimal('4.99'))
('BIKINI BORROWERS', Decimal('4.99'))
('BILKO ANONYMOUS', Decimal('4.99'))
('BIRCH ANTITRUST', Decimal('4.99'))
('BIRD INDEPENDENCE', Decimal('4.99'))
('BIRDS PERDITION', Decimal('4.99'))
('BLINDNESS GUN', Decimal('4.99'))
('BOILED DARES', Decimal('4.99'))
('BOOGIE AMELIE', Decimal('4.99')

### Vamos praticar

Use variáveis temporárias para encontrar o ator que mais participou de filmes.

In [27]:
db('''
DROP TABLE IF EXISTS temp_film_count;
''')

db('''
CREATE TEMPORARY TABLE temp_film_count
SELECT 
    actor_id,
    first_name,
    last_name,
    COUNT(film_id) AS num_films
FROM
    actor
    INNER JOIN film_actor USING (actor_id)
GROUP BY 
    actor_id;
''')

db('''
SELECT 
    MAX(num_films) INTO @max_num_films 
FROM
    temp_film_count;
''')

db('''
SELECT 
    actor_id, 
    first_name, 
    last_name, 
    @max_num_films
FROM 
    temp_film_count
WHERE
    num_films = @max_num_films;
''')

db('DROP TABLE temp_film_count;')

Executando query:
Executando query:
Executando query:
Executando query:
(107, 'GINA', 'DEGENERES', 42)
Executando query:


Verifique quantas vezes o filme "COWBOY DOOM" foi alugado usando a view `movie_count`

In [28]:
db('''
SELECT * FROM movie_count WHERE title = "COWBOY DOOM"
''')

Executando query:
('COWBOY DOOM', 18)


Registre um aluguel do filme "COWBOY DOOM", feito pelo funcionario "Jon Stephens" na loja id=1 para o cliente "JESSIE BANKS", na data '2019-01-01', com data de retorno '2019-01-08'

In [29]:
db('''
select title, inventory_id, store_id from 
film inner join inventory using (film_id) inner join store using (store_id)
where title = "COWBOY DOOM"
''')

Executando query:
('COWBOY DOOM', 845, 1)
('COWBOY DOOM', 846, 1)
('COWBOY DOOM', 4582, 1)


In [30]:
db('''
SELECT
    staff_id INTO @s
FROM 
    staff
WHERE 
    first_name = "Jon"
    AND last_name = "Stephens"
''')
db('''
SELECT
    customer_id INTO @c
FROM
    customer
WHERE
    first_name = "JESSIE"
    AND last_name = "BANKS"
''')
db('''
SELECT 
    inventory_id INTO @i
FROM 
    inventory
    INNER JOIN film USING (film_id)
WHERE
    title = "COWBOY DOOM"
    AND store_id = 1
LIMIT 1
''')
db('''
SELECT @s, @c, @i
''')

Executando query:
Executando query:
Executando query:
Executando query:
(2, 215, 845)


In [31]:
db('''
INSERT INTO rental (rental_date, inventory_id, customer_id, return_date, staff_id) VALUES ("2019-01-01", @i, @c, "2019-01-08", @s)
''')

Executando query:


Verifique usando a view que a contagem de alugueis do filme subiu.

In [32]:
db('''
SELECT * FROM movie_count WHERE title = "COWBOY DOOM"
''')

Executando query:
('COWBOY DOOM', 19)


## Operador `IN`

Suponha que desejamos listar todos os filmes dos 3 atores mais populares. Podemos começar listando os 3 atores mais populares:

In [33]:
db('''
SELECT 
    actor_id, first_name, last_name, COUNT(film_id) AS num_films
FROM
    actor
    INNER JOIN film_actor USING (actor_id)
GROUP BY 
    actor_id
ORDER BY 
    num_films DESC
LIMIT 3
''')

Executando query:
(107, 'GINA', 'DEGENERES', 42)
(102, 'WALTER', 'TORN', 41)
(198, 'MARY', 'KEITEL', 40)


Vamos criar uma tabela temporária para guardar a informação de `actor_id` desses atores:

In [34]:
db('DROP TABLE IF EXISTS temp_pop_actors;')
db('''
CREATE TEMPORARY TABLE temp_pop_actors
SELECT first_name, last_name, actor_id FROM
    actor
    INNER JOIN film_actor USING (actor_id)
GROUP BY 
    actor_id
ORDER BY 
    COUNT(film_id) DESC
LIMIT 3;
''')
db('''
SELECT * from temp_pop_actors
''')

Executando query:
Executando query:
Executando query:
('GINA', 'DEGENERES', 107)
('WALTER', 'TORN', 102)
('MARY', 'KEITEL', 198)


Por fim, vamos usar essa informação para listar os filmes dos atores populares:

In [35]:
db('''
SELECT DISTINCT
    title
FROM
    film
    INNER JOIN film_actor USING (film_id)
WHERE
    actor_id IN (SELECT actor_id FROM temp_pop_actors);
''')

Executando query:
('BED HIGHBALL',)
('CALENDAR GUNFIGHT',)
('CHAMBER ITALIAN',)
('CHAPLIN LICENSE',)
('CHARIOTS CONSPIRACY',)
('CLUELESS BUCKET',)
('COLDBLOODED DARLING',)
('CONEHEADS SMOOCHY',)
('DARKNESS WAR',)
('DEER VIRGINIAN',)
('DOGMA FAMILY',)
('ELEPHANT TROJAN',)
('EXCITEMENT EVE',)
('FRISCO FORREST',)
('GANDHI KWAI',)
('GOODFELLAS SALUTE',)
('GUNFIGHT MOON',)
('HALL CASSIDY',)
('HEARTBREAKERS BRIGHT',)
('HOOK CHARIOTS',)
('HYDE DOCTOR',)
('IMPACT ALADDIN',)
('INDIAN LOVE',)
('INTRIGUE WORST',)
('LICENSE WEEKEND',)
('LOUISIANA HARRY',)
('MAGNIFICENT CHITTY',)
('METAL ARMAGEDDON',)
('MIDNIGHT WESTWARD',)
('MOVIE SHAKESPEARE',)
('MUMMY CREATURES',)
('OPEN AFRICAN',)
('SEARCHERS WAIT',)
('SEVEN SWARM',)
('SIERRA DIVIDE',)
('SPIRITED CASUALTIES',)
('STORM HAPPINESS',)
('SUGAR WONKA',)
('TELEGRAPH VOYAGE',)
('TRAINSPOTTING STRANGERS',)
('WIFE TURN',)
('WINDOW SIDE',)
('AMELIE HELLFIGHTERS',)
('ARABIA DOGMA',)
('BANG KWAI',)
('CASABLANCA SUPER',)
('CASPER DRAGONFLY',)
('CROW GREASE',

Note o uso de *subqueries*!

Não se esqueça de limpar tudo no final!

In [36]:
db('DROP TABLE temp_pop_actors')

Executando query:


### Vamos praticar

Liste os atores que participaram dos 3 filmes mais rentáveis (aqueles que mais geraram receita para a locadora).

In [37]:
db('''
DROP TABLE IF EXISTS temp_film_total;
''')

db('''
CREATE TEMPORARY TABLE temp_film_total
SELECT 
    title, film_id, SUM(amount) as total_revenue
FROM
    film 
    INNER JOIN inventory USING (film_id)
    INNER JOIN rental USING (inventory_id)
    INNER JOIN payment USING (rental_id)
WHERE
    payment_date IS NOT NULL
GROUP BY
    film_id
ORDER BY 
    total_revenue DESC
LIMIT 3
''')
db('''
SELECT * from temp_film_total
''')

Executando query:
Executando query:
Executando query:
('TELEGRAPH VOYAGE', 879, Decimal('231.73'))
('WIFE TURN', 973, Decimal('223.69'))
('ZORRO ARK', 1000, Decimal('214.69'))


In [38]:
db('''
SELECT DISTINCT
    actor_id,
    first_name,
    last_name
FROM
    actor
    INNER JOIN film_actor USING (actor_id)
WHERE 
    film_id IN (SELECT film_id FROM temp_film_total)
ORDER BY
    first_name, last_name
''')

Executando query:
(111, 'CAMERON', 'ZELLWEGER')
(52, 'CARMEN', 'HUNT')
(107, 'GINA', 'DEGENERES')
(157, 'GRETA', 'MALDEN')
(155, 'IAN', 'TANDY')
(47, 'JULIA', 'BARRYMORE')
(178, 'LISA', 'MONROE')
(138, 'LUCILLE', 'DEE')
(174, 'MICHAEL', 'BENING')
(185, 'MICHAEL', 'BOLGER')
(166, 'NICK', 'DEGENERES')
(200, 'THORA', 'TEMPLE')
(158, 'VIVIEN', 'BASINGER')
(28, 'WOODY', 'HOFFMAN')


In [39]:
db('DROP TABLE temp_film_total')

Executando query:


## Subqueries

Os tópicos discutidos acima poderiam ser resolvidos, em grande parte, com subqueries. As subqueries são queries `SELECT` criadas dentro de outras queries. 

Poderíamos ter usado subqueries nos mesmos lugares onde usamos tabelas temporárias, nos tópicos acima. Quando a subquery pode ser transformada em uma tabela temporária independente, separada da query exterior, dizemos que a subquery é **não-correlacionada** com a query exterior.

Usar subqueries não-correlacionadas é um tópico controverso: podemos sempre usar uma tabela temporária ou, ás vezes, pensar em um `JOIN` simples. Aliás, muitas vezes o otimizador de queries do banco de dados transformará a subquery em `JOIN`, se isso for vantajoso em termos de desempenho.

Uma subquery que depende da query externa (e portanto não pode ser separada em uma tabela temporária independente) é chamada de **subquery correlacionada**. Nestes casos podemos ter que executar a subquery para cada linha da query exterior! 

### Vamos praticar

Vamos refazer a atividade dos filmes de atores populares, usando subqueries. 

Temos um problema: o MySQL não suporta ``LIMIT`` em subqueries com o operador ``IN``. Vamos investigar isso mais de perto. 

Em primeiro lugar faça uma tradução direta da implementação da atividade anterior trocando tabela temporária por subquery.

In [40]:
try:
    db('''
    SELECT 
        actor_id, first_name, last_name
    FROM
        actor
        INNER JOIN film_actor USING (actor_id)
    WHERE
        film_actor.film_id IN (
            SELECT 
                film_id
            FROM
                film
                INNER JOIN inventory USING (film_id)
                INNER JOIN rental USING (inventory_id)
                INNER JOIN payment USING (rental_id)
            WHERE
                payment_date IS NOT NULL
            GROUP BY film_id
            ORDER BY SUM(amount)
            LIMIT 3)
    ORDER BY actor_id
    ''')
except mysql.connector.ProgrammingError as e:
    print(f'ProgrammingError: {e}')

Executando query:
ProgrammingError: 1235 (42000): This version of MySQL doesn't yet support 'LIMIT & IN/ALL/ANY/SOME subquery'


Ok, apareceu o problema. Mas considere que o problema original não precisava de IN desde o começo! Construa essa solução.

In [41]:
db('''
SELECT DISTINCT
    actor_id, first_name, last_name
FROM
    actor
    INNER JOIN film_actor USING (actor_id)
    INNER JOIN (
        SELECT 
            film_id
        FROM
            film
            INNER JOIN inventory USING (film_id)
            INNER JOIN rental USING (inventory_id)
            INNER JOIN payment USING (rental_id)
        WHERE
            payment_date IS NOT NULL
        GROUP BY 
            film_id
        ORDER BY 
            SUM(amount) DESC
        LIMIT 3
    ) temp_film_total USING (film_id)
ORDER BY
    first_name, last_name
''')

Executando query:
(111, 'CAMERON', 'ZELLWEGER')
(52, 'CARMEN', 'HUNT')
(107, 'GINA', 'DEGENERES')
(157, 'GRETA', 'MALDEN')
(155, 'IAN', 'TANDY')
(47, 'JULIA', 'BARRYMORE')
(178, 'LISA', 'MONROE')
(138, 'LUCILLE', 'DEE')
(174, 'MICHAEL', 'BENING')
(185, 'MICHAEL', 'BOLGER')
(166, 'NICK', 'DEGENERES')
(200, 'THORA', 'TEMPLE')
(158, 'VIVIEN', 'BASINGER')
(28, 'WOODY', 'HOFFMAN')


## Desafio!

Faça uma lista de filmes que tenham mais de dois atores cujo nome inicia com a mesma letra do título do filme!

In [42]:
db('''
SELECT 
    film_id,
    title,
    COUNT(actor_id) AS cnt,
    GROUP_CONCAT(
        CONCAT(first_name, ' ', last_name)
        SEPARATOR ', '
    ) AS actors
FROM
    film
    INNER JOIN film_actor USING (film_id)
    INNER JOIN actor USING (actor_id)
WHERE
    LEFT(title, 1) = LEFT(first_name, 1)
GROUP BY 
    film_id
HAVING
    cnt > 2
ORDER BY 
    cnt DESC, 
    title ASC
''')

Executando query:
(194, 'CROW GREASE', 3, 'CAMERON STREEP, CHRIS BRIDGES, CAMERON ZELLWEGER')
(479, 'JEDI BENEATH', 3, 'JAMES PITT, JULIANNE DENCH, JANE JACKMAN')
(858, 'SUBMARINE BED', 3, 'SEAN GUINESS, SIDNEY CROWE, SCARLETT BENING')


## Conclusão

Façamos uma pausa para apreciar quão longe estamos: já conseguimos criar nossas tabelas, inserir informação, removê-la, atualizá-la, e consultar nossa base de maneiras bem sofisticadas! Vimos desde `SELECT` simples até buscas mais complexas envolvendo várias etapas de processamento para obter o dado desejado.

Por hoje é só, feche sua conexão e bom descanso... opa, que nada, vamos trabalhar no projeto! Mãos à obra!

In [43]:
connection.close()